In [13]:
! pip install transformers[torch] -U
! pip install datasets

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score

dataset = load_dataset("emotion")
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=512)  # Adjust max_length as needed

dataset_encoded = dataset.map(tokenize, batched=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [22]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,  # Adjust batch size according to your hardware
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)


In [23]:
# Define compute_metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {'accuracy': accuracy_score(labels, preds)}

In [11]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
)

trainer.train()

eval_result_finetuned = trainer.evaluate()
print(f"Fine-tuned model accuracy: {eval_result_finetuned['eval_accuracy']}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.248600,0.213025,0.929500
2,0.122600,0.146625,0.942500


Fine-tuned model accuracy: 0.9425


In [25]:
print(f"Fine-tuned model accuracy: {eval_result_finetuned['eval_accuracy']}")


Fine-tuned model accuracy: 0.9425
